In [1]:
from music21 import *
import collections
import numpy as np
import pandas as pd
import gzip
import random

music21: Certain music21 functions might need these optional packages: matplotlib, scipy;
                   if you run into errors, install them by following the instructions at
                   http://mit.edu/music21/doc/installing/installAdditional.html


In [31]:
from collections import Counter, defaultdict
from ea import initialisation, modelTrainer, musicPlayer, simulation, fitness, duration, individual, util
from music21 import corpus, note, duration, chord, pitch, interval
import pandas as pd
import random

In [ ]:
importlib.reload(initialisation)
importlib.reload(modelTrainer)
importlib.reload(musicPlayer)
importlib.reload(simulation)
importlib.reload(fitness)
importlib.reload(music21)
importlib.reload(constants)
importlib.reload(util)
importlib.reload(individual)
importlib.reload(fitness)

In [ ]:
wiki_corpus = corpus.corpora.LocalCorpus('wiki')
curr_corpus = corpus.corpora.CoreCorpus()
curr_corpus = curr_corpus.search('mozart', 'composer')

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
wiki_corpus = corpus.corpora.LocalCorpus('wiki')
curr_corpus = wiki_corpus.metadataBundle

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [10]:
# Initialize duration for individual first
duration_matrix = modelTrainer.train_duration_matrix(scores)

In [ ]:
individual: {
    sections: {
        'intro' : [Measure],
        'verse I' : [Measure],
        'chorus' : [Measure],
        'verse II' : [Measure],
        'chorus' : [Measure],
    }
    
    
}

In [ ]:
def initialize_by_duration(population_size):
    duration_population = []
    
    for i in range(population_size):
        for j in 4:
            

In [11]:
pitches = modelTrainer.get_pitches_per_score(scores)
pitches = list(map(lambda y: list(filter(lambda x: x != 'REST', y)), pitches))

In [12]:
pitch_matrix = modelTrainer.train_pitch_matrix(scores)

In [ ]:
pitch_matrix

In [16]:
symbols = []

for s in pitches:
    curr_symbols = []
    for i in range(2, len(s)):
        n1 = pitch.Pitch(s[i-2])
        n2 = pitch.Pitch(s[i-1])
        n3 = pitch.Pitch(s[i])
        # print(f'{n1}, {n2}, {n3}')
        
        impl_interval = n2.midi - n1.midi
        real_interval = n3.midi - n2.midi
        # print(f'ímpl int: {impl_interval}')
        # print(f'real int: {real_interval}')
        
        is_same_direction = (impl_interval <= 0 and real_interval <= impl_interval) or (impl_interval >= 0 and real_interval >= impl_interval) 
        is_small_impl_interval = abs(impl_interval) <= 5
        is_small_real_interval = abs(real_interval) <= 5
        # Small impl_interval
        if is_small_impl_interval:
            
            #small real_interval
            if is_small_real_interval:
                
                # Unchanging direction
                if is_same_direction:
                    new_symbol = "PV"
                # Changing direction
                else:
                    if n1 == n3:
                        new_symbol = "QE"
                    new_symbol = "QV"
            #Large real_interval
            else:
                # Unchanging direction
                if is_same_direction:
                    new_symbol = "PR"
                # Changing direction
                else:
                    new_symbol = "QR"
        # Large impl_interval
        else:
            #small real_interval
            if is_small_real_interval:
                # Unchanging direction
                if is_same_direction:
                    new_symbol = "PF"
                # Changing direction
                else:
                    if n1 == n3:
                        new_symbol = "QE"
                    new_symbol = "QF"
            #Large real_interval
            else:
                # Unchanging direction
                if is_same_direction:
                    new_symbol = "PL"
                # Changing direction
                else:
                    new_symbol = "QL"
        curr_symbols.append(new_symbol)
    symbols.append(curr_symbols)   

In [17]:
symbol_tuples = []
for j in symbols:
    for i in range(1, len(j)-1):

        prev = j[i-1]
        curr = j[i]
        symbol_tuples.append((prev,curr))

In [29]:
symbol_matrix

,PV,QV,QR,QF,QL,PR
QV,0.511166,0.454586,0.000000,0.371595,0.000000,0.000000
PV,0.432890,0.497210,0.000000,0.529183,0.000000,0.000000
QR,0.028423,0.018969,0.000000,0.021401,0.000000,0.000000
PR,0.027521,0.029235,0.000000,0.077821,0.000000,0.000000
QF,0.000000,0.000000,0.895455,0.000000,0.887755,0.787671
QL,0.000000,0.000000,0.104545,0.000000,0.112245,0.212329


In [21]:
matrix = defaultdict(lambda: defaultdict(lambda: 0))

for (i1,i2) in symbol_tuples:
    matrix[i1][i2] += 1
    
for i1 in matrix:
    total = float(sum(matrix[i1].values()))
    for i2 in matrix[i1]:
        matrix[i1][i2] /= total
    
pd_matrix = pd.DataFrame(matrix)
pd_matrix = pd_matrix.fillna(0)
symbol_matrix = pd_matrix

In [ ]:
pitch_matrix['C4']

In [34]:
generate_individual_by_symbols(symbol_matrix, pitch_matrix)

['C5', 'D5']
['C5', 'D5', 'F5']
['C5', 'D5', 'F5', 'D5']
['C5', 'D5', 'F5', 'D5', None]


b:\program files\python\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


KeyError: None

In [33]:
def generate_individual_by_symbols(symbol_matrix, pitch_matrix):
    individuals = []

    for i in range(0,2):
        symbols = []
        # Symbol size
        for i in range(0,20):
            if i == 0:
                s = initialisation.get_random_transition(symbol_matrix, None)
            else:
                s = initialisation.get_random_transition(symbol_matrix, symbols[-1]) 
            symbols.append(s)

        notes = []
        # Generate two initial notes
        notes.append('C5')
        initial_symbol = symbols[0]
        direction = random.choice(['UP'])
        if initial_symbol[1] == 'V' or initial_symbol[1] == 'R':
            second_note = sample_by_symbol('S', direction, notes[-1], pitch_matrix)
        else:
            second_note = sample_by_symbol('L', direction, notes[-1], pitch_matrix)
        notes.append(second_note)

        # Start generating the rest of the notes
        for i in range(2, len(symbols)):
            print(notes)
            s = symbols[i-2]
            
            direction = s[0]
            interval_size = s[1]
            
            n1 = notes[-1]
            n2 = notes[len(notes)-2]
            
            direction_explicit = get_direction(direction, n2, n1)
            interval_size_explicit = get_interval_size(interval_size)
            new_note = sample_by_symbol(
                interval_size_explicit, direction_explicit, notes[-1], pitch_matrix)
            notes.append(new_note)
            
        individuals.append(notes)
    return individuals

In [23]:
def get_direction(direction, prev_note1, prev_note2):
    if direction == 'Q':
        # If the direction goes up
        if pitch.Pitch(prev_note1) < pitch.Pitch(prev_note2):
            return "DOWN"
        # If the direction goes down
        else:
            return "UP"
    else:
        if pitch.Pitch(prev_note1) < pitch.Pitch(prev_note2):
            return "UP"
        else:
            return "DOWN"
def get_interval_size(symbol):
    if symbol == 'V' or symbol == 'F':
        return 'S'
    else:
        return 'L'

In [ ]:
sample_by_symbol('S', 'DOWN', 'G5', pitch_matrix)

In [ ]:
get_direction('Q', 'C5', 'G5')
get_interval_size('F')

In [24]:
def sample_by_symbol(interval_size, direction, prev_note, matrix):
    i = interval.Interval()
    intervals = []
    if interval_size == 'S':
        if direction == 'UP':
            r = range(0, 6)
        else:
            r = range(0, -6, -1)
    else:
        if direction == 'UP':
            r = range(6, 11)
        else:
            r = range(-6, -11, -1)
            
    for i in r:
        i = interval.Interval(i)
        inter = get_normal_pitch_name(i.transposePitch(pitch.Pitch(prev_note)))
        intervals.append(inter)
    return sample_random_pitch(matrix, intervals, prev_note)

In [26]:
def get_normal_pitch_name(pitch):
    pitch_name = pitch.nameWithOctave
    if '-' in pitch_name or '##' in pitch_name:
        return pitch.getEnharmonic().nameWithOctave
    else:
        return pitch_name

In [25]:
def sample_random_pitch(matrix, intervals, prev_note):
    total_count = 0
    m = defaultdict(lambda: defaultdict(lambda: 0))
    # Get the intervals and create a distribution
    for i in intervals:
        if i not in matrix[prev_note]:
           continue
        p = matrix[prev_note][i]
        # Create a temporary matrix saving the P and also sum up all probabilities
        m[prev_note][i] = p
        total_count += p

    # Normalize probabilities
    for i in intervals:
        m[prev_note][i] /= total_count

    # Select note in the constrainted space
    p_sum = 0
    rng = random.random()
    for i in intervals:
        v = m[prev_note][i]
        if p_sum < rng < p_sum + v:
            return i
        else:
            p_sum += v


In [ ]:
a = interval.Interval(1)
a.transposePitch(pitch.Pitch('C4'))

In [ ]:
individuals = generate_individual_by_symbols(symbol_matrix, pitch_matrix)
s = stream.Score(id = 'mainScore')
part = stream.Part(id='part1')
for i in individuals:
    
    counter = 1
    m = stream.Measure(counter)
    for p in i:
        n = note.Note(p)
        m.append(n)
    part.append(m)


In [ ]:
s.append(part)
s.show('musicxml')

In [ ]:
r = range(-1, -5, -1)
for i in r:
    print(i)